In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html
from pathlib import Path


def extract_endpoint(url):
    url = url.rstrip('/')
    url_parts = url.split('/')
    return f"{url_parts[-1]}.html"

def add_page_source(url):
    url=f"view-source:{url}"
    return url



with open("shark-tank.html") as file:
    contents = file.read()

soup = BeautifulSoup(contents,'html.parser')

all_div_tags = soup.find_all(name='div', class_='list-table-container')

df_list = []

for season in all_div_tags:
    season_number = int(season.get("data-number"))
    for episode in season.find_all(name='tr'):
        episode_number = int(episode.find(name='td').text)
        for show in episode.find_all("a"):
            company = html.unescape(show.getText())
            link = show.get("href")
            endpoint = extract_endpoint(link)
            page_source = add_page_source(link)
            df_list.append((season_number,episode_number,company,link,endpoint,page_source))





df = pd.DataFrame(df_list, columns=['Season','Episode','Company','Link',"Endpoint","Page Source"])
df = df.sort_values(by=["Season","Episode"])

df.to_csv("D:/Downloads/shark-tank.csv",index=False)

###################################################################################
full_df_list = []
for index,pitch in df.head(1).iterrows():
    air_date, business_pitch, entrepreneur, ask_money, ask_equity, deal, shark, business_status, valuation, website, summary = (None, None, None, None, None, None, None, None, None, None, None)

    # Get the current notebook directory (where the .ipynb file is located)
    notebook_dir = Path.cwd()
    file_path = notebook_dir / f"episode-html/{pitch.Endpoint}"

    # Open the file
    with open(file_path) as file:
        contents = file.read()

    soup = BeautifulSoup(contents,'html.parser')

    ep_table = soup.find(name='table',class_='ep-table')
    ep_table.find(name='tr',text="Air Date")
    ep_table = ep_table.find_all(name='tr')
    for section in ep_table:
        if section.find(name='th',text='Air Date'):
            air_date = section.find(name='td').text



    pi_table = soup.find(name='table',class_='pi-table')
    pi_table = pi_table.find(name='tbody')

    for section in pi_table.find_all(name='tr'):
        if section.find(name='th').text == "Business Pitch":
            business_pitch=section.find(name='td').text
        
        elif section.find(name='th').text == "Entrepreneur/Founder":
            entrepreneur=section.find(name='td').text

        elif section.find(name='th').text == "Asked For":
            ask=section.find(name='td').text
            ask_money = ask.split(" for ")[0]
            ask_equity = ask.split(" for ")[1]

        elif section.find(name='th').text == "Deal":
            deal=section.find(name='td').text

        elif section.find(name='th').text == "Shark":
            shark=section.find(name='td').text

    bi_table = soup.find(name='table',class_='bi-table')
    bi_table = bi_table.find(name='tbody')

    for section in bi_table.find_all(name='tr'):
        if section.find(name='th').text == "Business Status":
            business_status=section.find(name='td').text
        
        elif section.find(name='th').text == "Estimated Valuation":
            valuation=section.find(name='td').text

        elif section.find(name='th').text == "Website":
            website=section.find(name='td').text

    recap_summary = soup.find(name="ul",class_="recap-summary")
    recap_summary = recap_summary.find_all(name='li')
    summary = ""
    for bullet_point in recap_summary:
        try:
            summary = f"{summary}{bullet_point.find('span').get('class')[1]}{html.unescape(bullet_point.text)}\n"
        except AttributeError:
            summary = f"{summary}{bullet_point.text}\n"


    full_df_list.append((pitch.Season,pitch.Episode,pitch.Company,pitch.Link,pitch.Endpoint,pitch["Page Source"],air_date, business_pitch, entrepreneur, ask_money, ask_equity, deal, shark, business_status, valuation, website, summary))


full_df = pd.DataFrame(full_df_list, columns=['Season','Episode','Company','Link','Endpoint',"Page Source",'Air Date','Business Pitch','Entrepreneur','Ask Money','Ask Equity','Deal','Shark','Business Status','Valuation','Website','Summary'])
